Combining All History JSON Files

In [16]:
import pandas as pd

#combining total history
history1 = pd.read_json('history1.json')
history2 = pd.read_json('history2.json')
history3 = pd.read_json('history3.json')
complete = pd.concat([history1,history2,history3])
print(complete.columns)







Index(['ts', 'platform', 'ms_played', 'conn_country', 'ip_addr',
       'master_metadata_track_name', 'master_metadata_album_artist_name',
       'master_metadata_album_album_name', 'spotify_track_uri', 'episode_name',
       'episode_show_name', 'spotify_episode_uri', 'audiobook_title',
       'audiobook_uri', 'audiobook_chapter_uri', 'audiobook_chapter_title',
       'reason_start', 'reason_end', 'shuffle', 'skipped', 'offline',
       'offline_timestamp', 'incognito_mode'],
      dtype='object')


In [17]:
#Drop unecessary columns and rename original
complete = complete.drop(columns=['platform', 'ip_addr', 'audiobook_title', 'audiobook_uri', 'audiobook_chapter_uri', 'audiobook_chapter_title', 'offline', 'offline_timestamp'])
complete = complete.rename(columns={'ts':'exactTime', 'ms_played':'ms', 'conn_country':'country','master_metadata_track_name':'trackName',	'master_metadata_album_artist_name':'artistName','master_metadata_album_album_name':'albumName'	,'spotify_track_uri':'trackURI', 'episode_name':'episodeName','episode_show_name':'showName', 'spotify_episode_uri':'episodeURI','reason_start':'startReason','reason_end':'endReason','incognito_mode':'incognito'})




In [18]:
#convert ms to minutes
complete['duration'] = complete['ms']/60000
#extract specific times
complete['exactTime'] = pd.to_datetime(complete['exactTime'])
complete['hour'] = complete['exactTime'].dt.hour
complete['day'] = complete['exactTime'].dt.date
complete['month'] = complete['exactTime'].dt.month
complete['weekday'] = complete['exactTime'].dt.weekday
complete['year'] = complete['exactTime'].dt.year

complete.head() # complete['year'] = complete['year'].dt.year


,exactTime,ms,country,trackName,artistName,albumName,trackURI,episodeName,showName,episodeURI,...,endReason,shuffle,skipped,incognito,duration,hour,day,month,weekday,year
0,2021-04-09 05:01:10+00:00,305972,BZ,Hatsukoi,中野家の五つ子,Gotoubun no Katachi / Hatsukoi,spotify:track:6EnKh6904anMd7ACguRTZO,None,None,None,...,endplay,False,False,False,5.099533,5,2021-04-09,4,4,2021
1,2021-04-09 05:01:10+00:00,305972,BZ,Hatsukoi,中野家の五つ子,Gotoubun no Katachi / Hatsukoi,spotify:track:6EnKh6904anMd7ACguRTZO,None,None,None,...,fwdbtn,False,False,False,5.099533,5,2021-04-09,4,4,2021
2,2021-04-09 05:06:03+00:00,290940,BZ,Gobun no Ichi,中野家の五つ子,Gotoubun no Kimochi,spotify:track:10qqd7XJ1rVvF8slz7QASv,None,None,None,...,trackdone,False,False,False,4.849000,5,2021-04-09,4,4,2021
3,2021-04-09 05:06:03+00:00,290940,BZ,Gobun no Ichi,中野家の五つ子,Gotoubun no Kimochi,spotify:track:10qqd7XJ1rVvF8slz7QASv,None,None,None,...,trackdone,False,False,False,4.849000,5,2021-04-09,4,4,2021
4,2021-11-23 04:09:43+00:00,52083,BZ,Asgore's Theme,Caleb Hyles,Asgore's Theme,spotify:track:4AZpCL57dG19fsk23YLS1K,None,None,None,...,fwdbtn,True,False,False,0.868050,4,2021-11-23,11,1,2021


In [19]:
#Split up dataframe into tracks and episodes/podcasts

episodes = complete[complete['episodeName'].notnull()] #save rows where there's an episodeName
tracks = complete[complete['trackName'].notnull()] #save rows where there's a trackName
print(len(episodes))
print(len(tracks))
print(len(complete))
#tracks = complete[~complete['trackName'].notnull()]


77
38795
38882


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time




HTTP Error for GET to https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ/related-artists with Params: {} returned 404 due to Not Found


Artist ID: 1Xyo4u8uXC1ZmMpatF05PJ
[]
{'artists': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=1&query=The%20Weeknd&type=artist', 'limit': 1, 'next': 'https://api.spotify.com/v1/search?offset=1&limit=1&query=The%20Weeknd&type=artist', 'offset': 0, 'previous': None, 'total': 828, 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ'}, 'followers': {'href': None, 'total': 111765896}, 'genres': [], 'href': 'https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ', 'id': '1Xyo4u8uXC1ZmMpatF05PJ', 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eb9e528993a2820267b97f6aae', 'height': 640, 'width': 640}, {'url': 'https://i.scdn.co/image/ab676161000051749e528993a2820267b97f6aae', 'height': 320, 'width': 320}, {'url': 'https://i.scdn.co/image/ab6761610000f1789e528993a2820267b97f6aae', 'height': 160, 'width': 160}], 'name': 'The Weeknd', 'popularity': 96, 'type': 'artist', 'uri': 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ'}]}}


SpotifyException: http status: 404, code:-1 - https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ/related-artists:
 Not Found, reason: None